## Minneapolis, MN Simulations
Need to add Scenario field to CSV files.<br>
8760 rows per simulation<br>
49 files 

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 0.83
elec = 0.1314

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

---

## Combine v9.5 CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/95/msp/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data95 = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/95/msp/" + file)
    all_data95 = pd.concat([all_data95, df])
    
all_data95

# 429240  rows, 214 columns

,Scenario,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Direct Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Exterior Horizontal Sky Illuminance [lux](Hourly),Environment:Site Exterior Horizontal Beam Illuminance [lux](Hourly),Environment:Site Exterior Beam Normal Illuminance [lux](Hourly),Environment:Site Sky Diffuse Solar Radiation Luminous Efficacy [lum/W](Hourly),Environment:Site Beam Solar Radiation Luminous Efficacy [lum/W](Hourly),...,WINDOW_SDL2_2_BOT.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),LIVING_UNIT1:Zone Operative Temperature [C](Hourly),ATTIC_UNIT1:Zone Operative Temperature [C](Hourly),SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly)
0,1,01/01 01:00:00,-6.6500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.473440,-2.452388,378211.7161,24412005.56,0.0
1,1,01/01 02:00:00,-7.9875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.442552,-4.927196,424942.6963,27428297.48,0.0
2,1,01/01 03:00:00,-7.4250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.423690,-5.761685,436672.6658,28185418.61,0.0
3,1,01/01 04:00:00,-6.8875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.406847,-5.776410,436173.5662,28153203.79,0.0
4,1,01/01 05:00:00,-6.3250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.393771,-5.509716,446361.0315,28810762.64,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,49,12/31 20:00:00,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.213657,0.941925,368766.0565,23802327.21,0.0
8756,49,12/31 21:00:00,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.211561,0.902576,368626.0958,23793293.32,0.0
8757,49,12/31 22:00:00,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.195448,0.867043,380687.2974,24571794.11,0.0
8758,49,12/31 23:00:00,-0.3750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.163501,0.745551,396022.1627,25561596.39,0.0


In [5]:
# Get wanted columns
all_data95_1 = all_data95[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

all_data95_1

,Scenario,Date/Time,LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly),SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly)
0,1,01/01 01:00:00,0.0,378211.7161,24412005.56,0.0
1,1,01/01 02:00:00,0.0,424942.6963,27428297.48,0.0
2,1,01/01 03:00:00,0.0,436672.6658,28185418.61,0.0
3,1,01/01 04:00:00,0.0,436173.5662,28153203.79,0.0
4,1,01/01 05:00:00,0.0,446361.0315,28810762.64,0.0
...,...,...,...,...,...,...
8755,49,12/31 20:00:00,0.0,368766.0565,23802327.21,0.0
8756,49,12/31 21:00:00,0.0,368626.0958,23793293.32,0.0
8757,49,12/31 22:00:00,0.0,380687.2974,24571794.11,0.0
8758,49,12/31 23:00:00,0.0,396022.1627,25561596.39,0.0


In [6]:
# all_data95_1 = all_data95_1.dropna().reset_index(drop=True)

# all_data95_1

# #438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [7]:
# Rename Columns
all_data95_2 = all_data95_1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data95_2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [8]:
# Copy dataframe to modify and leave original df intact
get_annual95 = all_data95_2.copy(deep=True)

#get_annual

In [9]:
# Convert Joules to KBtu in dataframe
get_annual95["FanEnergy(kBtu)"] = get_annual95["FanEnergy[J](Hourly)"] * .00000094781712
get_annual95["HeatingEnergy(kBtu)"] = get_annual95["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual95["CoolingEnergy(kBtu)"] = get_annual95["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual95

In [10]:
#list(get_annual95.columns.values)

In [11]:
# Drop columns
get_annual95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual95

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [12]:
# Get annual costs
get_annual95_costs = all_data95_2.copy(deep=True)

In [13]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual95_costs["FanCost($)"] = (get_annual95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual95_costs["HeatingCost($)"] = ((get_annual95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual95_costs["CoolingCost($)"] = (get_annual95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs95

In [14]:
# Drop columns
get_annual95_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs95

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [15]:
# Combine data for energy and cost
allAnnualData95 = pd.merge(get_annual95, get_annual95_costs)

#allAnnualData95

In [16]:
# Add energy and cost totals
totalEnergy95 = allAnnualData95["FanEnergy(kBtu)"] + allAnnualData95["HeatingEnergy(kBtu)"] + allAnnualData95["CoolingEnergy(kBtu)"]
allAnnualData95["AnnualEnergy(kBtu)"] = totalEnergy95

totalCost95 = allAnnualData95["FanCost($)"] + allAnnualData95["HeatingCost($)"] + allAnnualData95["CoolingCost($)"]
allAnnualData95["AnnualCost($)"] = totalCost95

#allAnnualData95

In [17]:
#list(allAnnualData95.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [18]:
# Group by scenario
annualData95 = allAnnualData95.groupby(["Scenario"], as_index=True)

#annualData95.sum().round(2)

In [19]:
annualDataFinal95 = annualData95.sum().round(2)

#annualDataFinal95

In [20]:
# Export grouped file to csv
annualDataFinal95.to_csv("../Regression_Scraper_Output/msp_annual95_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [21]:
# Copy dataframe to modify and leave original df intact
get_monthly95 = all_data95_2.copy(deep=True)

#get_monthly95

In [22]:
# Convert Joules to kWh and therms in new dataframe
get_monthly95["FanEnergy(kWh)"] = get_monthly95["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly95["HeatingEnergy(therms)"] = get_monthly95["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly95["CoolingEnergy(kWh)"] = get_monthly95["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly95

In [23]:
# Drop columns
get_monthly95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly95

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [24]:
# Copy dataframe to modify and leave original df intact
get_monthly95_costs = all_data95_2.copy(deep=True)

#get_monthly95

In [25]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly95_costs["FanCost($)"] = (get_monthly95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly95_costs["HeatingCost($)"] = ((get_monthly95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly95_costs["CoolingCost($)"] = (get_monthly95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly95_costs

In [26]:
# Drop columns
get_monthly95_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly95_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [27]:
# Combine data for energy and cost
allMonthlyData95 = pd.merge(get_monthly95, get_monthly95_costs, how="inner")

#allMonthlyData95

In [28]:
# Add cost totals, don't total energy...in different units
totalCostMonthly95 = allMonthlyData95["FanCost($)"] + allMonthlyData95["HeatingCost($)"] + allMonthlyData95["CoolingCost($)"]
allMonthlyData95["MonthlyCost($)"] = totalCostMonthly95

#allMonthlyData95

In [29]:
allMonthlyData95["Month"] = allMonthlyData95.Date_Time.str.slice(0,3)

allMonthlyData95

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanCost($),HeatingCost($),CoolingCost($),MonthlyCost($),Month
0,1,01/01 01:00:00,0.0,0.105059,0.231381,0.0,0.013805,0.192046,0.0,0.205851,01
1,1,01/01 02:00:00,0.0,0.118040,0.259970,0.0,0.015510,0.215775,0.0,0.231286,01
2,1,01/01 03:00:00,0.0,0.121298,0.267146,0.0,0.015939,0.221731,0.0,0.237670,01
3,1,01/01 04:00:00,0.0,0.121159,0.266841,0.0,0.015920,0.221478,0.0,0.237398,01
4,1,01/01 05:00:00,0.0,0.123989,0.273073,0.0,0.016292,0.226651,0.0,0.242943,01
...,...,...,...,...,...,...,...,...,...,...,...
429235,49,12/31 20:00:00,0.0,0.102435,0.225603,0.0,0.013460,0.187250,0.0,0.200710,12
429236,49,12/31 21:00:00,0.0,0.102396,0.225517,0.0,0.013455,0.187179,0.0,0.200634,12
429237,49,12/31 22:00:00,0.0,0.105746,0.232896,0.0,0.013895,0.193303,0.0,0.207198,12
429238,49,12/31 23:00:00,0.0,0.110006,0.242277,0.0,0.014455,0.201090,0.0,0.215545,12


## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [30]:
# Group by scenario and date/time for export to graphing
MonthlyData95_2 = allMonthlyData95.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData95_2.sum()

In [31]:
MonthlyDataFinal95 = MonthlyData95_2.sum()

#MonthlyDataFinal95

In [32]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal95.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [33]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal95.round(2).to_csv("../Regression_Scraper_Output/msp_monthly95_data.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [34]:
# Copy dataframe to modify and leave original df intact
all_data95_3 = all_data95_1.copy(deep=True)

#all_data95_3

In [35]:
# Get wanted columns
# Include hourly illumance
all_data95_3 = all_data95_3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data95_3

In [36]:
# Rename Columns
get_hourly95 = all_data95_3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly95

In [37]:
# Convert Joules to kBtus in new dataframe
get_hourly95["FanEnergy(kBtu)"] = get_hourly95["FanEnergy[J](Hourly)"] * .00000094781712
get_hourly95["HeatingEnergy(kBtu)"] = get_hourly95["HeatingEnergy[J](Hourly)"] * .00000094781712
get_hourly95["CoolingEnergy(kBtu)"] = get_hourly95["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_hourly95

In [38]:
# Drop columns
get_hourly95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly95

In [39]:
#list(get_hourly95.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [40]:
# Copy dataframe to modify and leave original df intact
get_hourly95_costs = all_data95_3.copy(deep=True)

#get_hourly95_costs

In [41]:
# Rename Columns
get_hourly95_costs = get_hourly95_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly95_costs

In [42]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly95_costs["FanCost($)"] = (get_hourly95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly95_costs["HeatingCost($)"] = ((get_hourly95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly95_costs["CoolingCost($)"] = (get_hourly95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

##get_hourly95_costs

In [43]:
# Drop columns
get_hourly95_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly95_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Extract Date_Time<br>
Export combine file

In [44]:
# Combine data for energy and cost
allHourlyData95 = pd.merge(get_hourly95, get_hourly95_costs, how="inner")

#allHourlyData95

In [45]:
#Extract hour, day, month.
allHourlyData95["Month"] = allHourlyData95.Date_Time.str.slice(0,3)
allHourlyData95["Day"] = allHourlyData95.Date_Time.str.slice(4,6)
allHourlyData95["Hour"] = allHourlyData95["Date_Time"].astype(str).str[8:16]

allHourlyData95

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanCost($),HeatingCost($),CoolingCost($),Month,Day,Hour
0,1,01/01 01:00:00,0.0,0.358476,23.138117,0.0,0.013805,0.192046,0.0,01,01,01:00:00
1,1,01/01 02:00:00,0.0,0.402768,25.997010,0.0,0.015510,0.215775,0.0,01,01,02:00:00
2,1,01/01 03:00:00,0.0,0.413886,26.714622,0.0,0.015939,0.221731,0.0,01,01,03:00:00
3,1,01/01 04:00:00,0.0,0.413413,26.684089,0.0,0.015920,0.221478,0.0,01,01,04:00:00
4,1,01/01 05:00:00,0.0,0.423069,27.307334,0.0,0.016292,0.226651,0.0,01,01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
429235,49,12/31 20:00:00,0.0,0.349523,22.560253,0.0,0.013460,0.187250,0.0,12,31,20:00:00
429236,49,12/31 21:00:00,0.0,0.349390,22.551691,0.0,0.013455,0.187179,0.0,12,31,21:00:00
429237,49,12/31 22:00:00,0.0,0.360822,23.289567,0.0,0.013895,0.193303,0.0,12,31,22:00:00
429238,49,12/31 23:00:00,0.0,0.375357,24.227719,0.0,0.014455,0.201090,0.0,12,31,23:00:00


In [46]:
# Add energy and cost totals

totalEnergyhourly95 = allHourlyData95["FanEnergy(kBtu)"] + allHourlyData95["HeatingEnergy(kBtu)"] + allHourlyData95["CoolingEnergy(kBtu)"]
totalCosthourly95 = allHourlyData95["FanCost($)"] + allHourlyData95["HeatingCost($)"] + allHourlyData95["CoolingCost($)"]
allHourlyData95["HourlyEnergy(kBtu)"] = totalEnergyhourly95
allHourlyData95["HourlyCost($)"] = totalCosthourly95

#allHourlyData95

In [47]:
# Export grouped file to csv (used before grouped by month)
allHourlyData95.round(2).to_csv("../Regression_Scraper_Output/msp_hourly95_data.csv", header=True, index=True)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## ---

## Combine v9.2 CSV Files in Directory

In [48]:
# Create variable for files in directory
files = [f for f in os.listdir("data/92/hou/") if f.endswith(".csv")]

# files

In [49]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data92 = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/92/hou/" + file)
    all_data92 = pd.concat([all_data92, df])
    
all_data92

# 429240  rows, 214 columns

,Scenario,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Direct Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Exterior Horizontal Sky Illuminance [lux](Hourly),Environment:Site Exterior Horizontal Beam Illuminance [lux](Hourly),Environment:Site Exterior Beam Normal Illuminance [lux](Hourly),Environment:Site Sky Diffuse Solar Radiation Luminous Efficacy [lum/W](Hourly),Environment:Site Beam Solar Radiation Luminous Efficacy [lum/W](Hourly),...,WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),LIVING_UNIT1:Zone Operative Temperature [C](Hourly),ATTIC_UNIT1:Zone Operative Temperature [C](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly)
0,1,01/01 01:00:00,2.9875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.647216,5.595716,1.0,1.0,12.300694,21.613215,483178.06050,1.469858e+07,0.0
1,1,01/01 02:00:00,2.8000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.556717,5.004605,1.0,1.0,12.300694,21.613215,530474.06590,1.613735e+07,0.0
2,1,01/01 03:00:00,2.4250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.495148,4.631407,1.0,1.0,12.300694,21.613215,572067.20310,1.740264e+07,0.0
3,1,01/01 04:00:00,1.8875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.451549,4.200492,1.0,1.0,12.300694,21.613215,612237.15210,1.862463e+07,0.0
4,1,01/01 05:00:00,1.7000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.424331,3.806778,1.0,1.0,12.300694,21.613215,631400.91020,1.920761e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,49,12/31 20:00:00,11.1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,23.521424,15.380179,1.0,1.0,12.613681,21.710241,0.00000,0.000000e+00,0.0
8756,49,12/31 21:00:00,10.0375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,22.726967,13.297838,1.0,1.0,12.613681,21.710241,0.00000,0.000000e+00,0.0
8757,49,12/31 22:00:00,8.0250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,21.781244,11.686825,1.0,1.0,12.613681,21.710241,24921.83745,7.581377e+05,0.0
8758,49,12/31 23:00:00,6.8875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,21.208649,10.088615,1.0,1.0,12.613681,21.710241,208511.00970,6.343034e+06,0.0


In [50]:
# Get wanted columns
all_data92_1 = all_data92[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

all_data92_1

,Scenario,Date/Time,LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly),SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly)
0,1,01/01 01:00:00,0.0,483178.06050,1.469858e+07,0.0
1,1,01/01 02:00:00,0.0,530474.06590,1.613735e+07,0.0
2,1,01/01 03:00:00,0.0,572067.20310,1.740264e+07,0.0
3,1,01/01 04:00:00,0.0,612237.15210,1.862463e+07,0.0
4,1,01/01 05:00:00,0.0,631400.91020,1.920761e+07,0.0
...,...,...,...,...,...,...
8755,49,12/31 20:00:00,0.0,0.00000,0.000000e+00,0.0
8756,49,12/31 21:00:00,0.0,0.00000,0.000000e+00,0.0
8757,49,12/31 22:00:00,0.0,24921.83745,7.581377e+05,0.0
8758,49,12/31 23:00:00,0.0,208511.00970,6.343034e+06,0.0


In [51]:
# all_data92_1 = all_data92_1.dropna().reset_index(drop=True)

# all_data92_1

# #438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [52]:
# Rename Columns
all_data92_2 = all_data92_1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data92_2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [53]:
# Copy dataframe to modify and leave original df intact
get_annual92 = all_data92_2.copy(deep=True)

#get_annual92

In [54]:
# Convert Joules to KBtu in dataframe
get_annual92["FanEnergy(kBtu)"] = get_annual92["FanEnergy[J](Hourly)"] * .00000094781712
get_annual92["HeatingEnergy(kBtu)"] = get_annual92["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual92["CoolingEnergy(kBtu)"] = get_annual92["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual92

In [55]:
#list(get_annual92.columns.values)

In [56]:
# Drop columns
get_annual92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual92

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [57]:
# Get annual costs
get_annual92_costs = all_data92_2.copy(deep=True)

In [58]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual92_costs["FanCost($)"] = (get_annual92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual92_costs["HeatingCost($)"] = ((get_annual92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual92_costs["CoolingCost($)"] = (get_annual92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual92_costs

In [59]:
# Drop columns
get_annual92_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual92_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [60]:
# Combine data for energy and cost
allAnnualData92 = pd.merge(get_annual92, get_annual92_costs)

#allAnnualData2

In [61]:
# Add energy and cost totals
totalEnergy92 = allAnnualData92["FanEnergy(kBtu)"] + allAnnualData92["HeatingEnergy(kBtu)"] + allAnnualData92["CoolingEnergy(kBtu)"]
allAnnualData92["AnnualEnergy(kBtu)"] = totalEnergy92

totalCost92 = allAnnualData92["FanCost($)"] + allAnnualData92["HeatingCost($)"] + allAnnualData92["CoolingCost($)"]
allAnnualData92["AnnualCost($)"] = totalCost92

#allAnnualData92

In [62]:
#list(allAnnualData92.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [63]:
# Group by scenario
annualData92 = allAnnualData92.groupby(["Scenario"], as_index=True)

#annualData92.sum().round(2)

In [64]:
annualDataFinal92 = annualData92.sum().round(2)

#annualDataFinal92

In [65]:
# Export grouped file to csv
annualDataFinal92.to_csv("../Regression_Scraper_Output/msp_annual92_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [66]:
# Copy dataframe to modify and leave original df intact
get_monthly92 = all_data92_2.copy(deep=True)

#get_monthly92

In [67]:
# Convert Joules to kWh and therms in new dataframe
get_monthly92["FanEnergy(kWh)"] = get_monthly92["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly92["HeatingEnergy(therms)"] = get_monthly92["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly92["CoolingEnergy(kWh)"] = get_monthly92["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly92

In [68]:
# Drop columns
get_monthly92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly92

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [69]:
# Copy dataframe to modify and leave original df intact
get_monthly92_costs = all_data92_2.copy(deep=True)

#get_monthly92

In [70]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly92_costs["FanCost($)"] = (get_monthly92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly92_costs["HeatingCost($)"] = ((get_monthly92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly92_costs["CoolingCost($)"] = (get_monthly92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly92_costs

In [71]:
# Drop columns
get_monthly92_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly92_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [72]:
# Combine data for energy and cost
allMonthlyData92 = pd.merge(get_monthly92, get_monthly92_costs, how="inner")

#allMonthlyData92

In [73]:
# Add cost totals, don't total energy...in different units
totalCostMonthly92 = allMonthlyData92["FanCost($)"] + allMonthlyData92["HeatingCost($)"] + allMonthlyData92["CoolingCost($)"]
allMonthlyData92["MonthlyCost($)"] = totalCostMonthly92

#allMonthlyData92

In [74]:
allMonthlyData92["Month"] = allMonthlyData92.Date_Time.str.slice(0,3)

allMonthlyData92

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanCost($),HeatingCost($),CoolingCost($),MonthlyCost($),Month
0,1,01/01 01:00:00,0.0,0.134216,0.139316,0.0,0.017636,0.115632,0.0,0.133268,01
1,1,01/01 02:00:00,0.0,0.147354,0.152953,0.0,0.019362,0.126951,0.0,0.146313,01
2,1,01/01 03:00:00,0.0,0.158908,0.164945,0.0,0.020880,0.136905,0.0,0.157785,01
3,1,01/01 04:00:00,0.0,0.170066,0.176527,0.0,0.022347,0.146518,0.0,0.168864,01
4,1,01/01 05:00:00,0.0,0.175389,0.182053,0.0,0.023046,0.151104,0.0,0.174150,01
...,...,...,...,...,...,...,...,...,...,...,...
429235,49,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
429236,49,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
429237,49,12/31 22:00:00,0.0,0.006923,0.007186,0.0,0.000910,0.005964,0.0,0.006874,12
429238,49,12/31 23:00:00,0.0,0.057920,0.060120,0.0,0.007611,0.049900,0.0,0.057511,12


## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [75]:
# Group by scenario and date/time for export to graphing
MonthlyData92_2 = allMonthlyData92.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData92_2.sum()

In [76]:
MonthlyDataFinal92 = MonthlyData92_2.sum()

#MonthlyDataFinal92

In [77]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal92.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [78]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal92.round(2).to_csv("../Regression_Scraper_Output/msp_monthly92_data.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [79]:
# Copy dataframe to modify and leave original df intact
all_data92_3 = all_data92_1.copy(deep=True)

#all_data92_3

In [80]:
# Get wanted columns
# Include hourly illumance
all_data92_3 = all_data92_3[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]
#all_data92_3

In [81]:
# Rename Columns
get_hourly92 = all_data92_3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly92

In [82]:
# Convert Joules to kBtus in new dataframe
get_hourly92["FanEnergy(kBtu)"] = get_hourly92["FanEnergy[J](Hourly)"] * .00000094781712
get_hourly92["HeatingEnergy(kBtu)"] = get_hourly92["HeatingEnergy[J](Hourly)"] * .00000094781712
get_hourly92["CoolingEnergy(kBtu)"] = get_hourly92["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_hourly92

In [83]:
# Drop columns
get_hourly92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly92

In [84]:
#list(get_hourly92.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [85]:
# Copy dataframe to modify and leave original df intact
get_hourly92_costs = all_data92_3.copy(deep=True)

#get_hourly92_costs

In [86]:
# Rename Columns
get_hourly92_costs = get_hourly92_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly92_costs

In [87]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly92_costs["FanCost($)"] = (get_hourly92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly92_costs["HeatingCost($)"] = ((get_hourly92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly92_costs["CoolingCost($)"] = (get_hourly92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

##get_hourly92_costs

In [88]:
# Drop columns
get_hourly92_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly92_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Extract Date_Time<br>
Export combine file

In [89]:
# Combine data for energy and cost
allHourlyData92 = pd.merge(get_hourly92, get_hourly92_costs, how="inner")

#allHourlyData92

In [90]:
#Extract hour, day, month.
allHourlyData92["Month"] = allHourlyData92.Date_Time.str.slice(0,3)
allHourlyData92["Day"] = allHourlyData92.Date_Time.str.slice(4,6)
allHourlyData92["Hour"] = allHourlyData92["Date_Time"].astype(str).str[8:16]

allHourlyData92

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanCost($),HeatingCost($),CoolingCost($),Month,Day,Hour
0,1,01/01 01:00:00,0.0,0.457964,13.931562,0.0,0.017636,0.115632,0.0,01,01,01:00:00
1,1,01/01 02:00:00,0.0,0.502792,15.295256,0.0,0.019362,0.126951,0.0,01,01,02:00:00
2,1,01/01 03:00:00,0.0,0.542215,16.494519,0.0,0.020880,0.136905,0.0,01,01,03:00:00
3,1,01/01 04:00:00,0.0,0.580289,17.652746,0.0,0.022347,0.146518,0.0,01,01,04:00:00
4,1,01/01 05:00:00,0.0,0.598453,18.205298,0.0,0.023046,0.151104,0.0,01,01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
429235,49,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,12,31,20:00:00
429236,49,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,12,31,21:00:00
429237,49,12/31 22:00:00,0.0,0.023621,0.718576,0.0,0.000910,0.005964,0.0,12,31,22:00:00
429238,49,12/31 23:00:00,0.0,0.197630,6.012036,0.0,0.007611,0.049900,0.0,12,31,23:00:00


In [91]:
# Add energy and cost totals

totalEnergyhourly92 = allHourlyData92["FanEnergy(kBtu)"] + allHourlyData92["HeatingEnergy(kBtu)"] + allHourlyData92["CoolingEnergy(kBtu)"]
totalCosthourly92 = allHourlyData92["FanCost($)"] + allHourlyData92["HeatingCost($)"] + allHourlyData92["CoolingCost($)"]
allHourlyData92["HourlyEnergy(kBtu)"] = totalEnergyhourly92
allHourlyData92["HourlyCost($)"] = totalCosthourly92

#allHourlyData92

In [92]:
# Export grouped file to csv (used before grouped by month)
allHourlyData92.round(2).to_csv("../Regression_Scraper_Output/msp_hourly92_data.csv", header=True, index=True)